In [1]:
%load_ext autoreload
%autoreload 2

from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn


from model import *
import pickle


In [2]:
#torch.set_grad_enabled(False)

In [3]:
# loading image data
noisy_imgs_1, noisy_imgs_2 = torch.load('../../data/train_data.pkl')
noisy_imgs, clean_imgs = torch.load('../../data/val_data.pkl')

train_input, train_target = noisy_imgs_1.float()/255.0, noisy_imgs_2.float()/255.0 
test_input, test_target = noisy_imgs.float()/255.0, clean_imgs.float()/255.0

In [4]:
# take batch of size 1
input_batch = train_input[:10,:,:,:]
target_batch = train_target[:10,:,:,:]
input_batch.shape

torch.Size([10, 3, 32, 32])

In [5]:
# create some tensors for simple tests
x = torch.randn((1, 3, 4, 4))

# y = torch.ones(x.shape)
y = torch.ones((1, 3, 4, 4))

### Testing our Conv2d Layer 

Forward pass

In [6]:
input_batch.shape, target_batch.shape

(torch.Size([10, 3, 32, 32]), torch.Size([10, 3, 32, 32]))

In [7]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2, stride=2)
output = my_conv.forward(input_batch)
output.shape

torch.Size([10, 4, 16, 16])

In [8]:
torch.testing.assert_allclose(my_conv.forward(input_batch), torch.nn.functional.conv2d(input_batch, my_conv.weight, my_conv.bias, stride=2))

Backward pass

In [12]:
back = my_conv.backward(output)
back.shape

torch.Size([10, 3, 32, 32])

### Testing our Upsampling (TransposeConv2d) layer

Forward pass

In [13]:
input_batch.shape

torch.Size([10, 3, 32, 32])

In [14]:
my_t_conv = Upsampling(3, 5, 2, stride=2)
upsampled = my_t_conv.forward(input_batch)
upsampled.shape

torch.Size([10, 5, 64, 64])

In [15]:
torch.testing.assert_allclose(my_t_conv.forward(input_batch), torch.nn.functional.conv_transpose2d(input_batch, my_t_conv.weight, my_t_conv.bias, stride=2))

Backward pass

In [16]:
back_t = my_t_conv.backward(upsampled)
back_t.shape

torch.Size([10, 3, 32, 32])

### Sequential model testing

In [18]:
model = Sequential(Conv2d(3,4,2), ReLU())
forward_pass = model.forward(x)
backward_pass = model.backward(forward_pass)
forward_pass.shape, backward_pass.shape

(torch.Size([1, 4, 3, 3]), torch.Size([1, 3, 4, 4]))

### Testing model

In [24]:
train_input_batch = train_input[:100, :, :, :]
train_target_batch = train_target[:100, :, :, :]
train_input_batch.shape, train_target_batch.shape

(torch.Size([100, 3, 32, 32]), torch.Size([0, 3, 32, 32]))

Train the model 

In [20]:
model = Model()
model.train(train_input_batch, train_target_batch, 50)

0 iteration: loss=0.7453855872154236
1 iteration: loss=0.7247834205627441
2 iteration: loss=0.7232609391212463
3 iteration: loss=0.7229578495025635
4 iteration: loss=0.7227702736854553
5 iteration: loss=0.7226124405860901
6 iteration: loss=0.722456693649292
7 iteration: loss=0.7223063707351685
8 iteration: loss=0.7221737504005432
9 iteration: loss=0.7220564484596252
10 iteration: loss=0.72194504737854
11 iteration: loss=0.721835732460022
12 iteration: loss=0.7217323780059814
13 iteration: loss=0.7216359376907349
14 iteration: loss=0.7215480208396912
15 iteration: loss=0.7214692831039429
16 iteration: loss=0.7213974595069885
17 iteration: loss=0.7213294506072998
18 iteration: loss=0.7212637066841125
19 iteration: loss=0.7211973667144775
20 iteration: loss=0.721129298210144
21 iteration: loss=0.7210604548454285
22 iteration: loss=0.7209893465042114
23 iteration: loss=0.720916748046875
24 iteration: loss=0.7208458185195923
25 iteration: loss=0.7207791805267334
26 iteration: loss=0.7207168

Prediction and psnr score

In [21]:
def psnr(denoised,ground_truth):
    mse=torch.mean((denoised-ground_truth)**2)
    return  - 10 * torch.log10(((denoised-ground_truth) ** 2).mean((1,2,3))).mean()

In [22]:
prediction = model.predict(test_input)
psnr(prediction, test_target)

tensor(12.7146)

Save the model

In [129]:
pickle.dump(model, open('bestmodel.pkl', 'wb'))